# Grade NBA Predictions

### Module Imports

In [1]:
from os import path
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
from datetime import date

### Global Variables and Settings

In [2]:
# Display all columns of DataFrames
pd.options.display.max_columns = None

# Print all output in a cell not just the last piece of output
InteractiveShell.ast_node_interactivity = "all"

# Location of sports data
DATA_DIR = 'C:\\Users\\Harry\\Documents\\LTCWFF\\ltcwff_files\\data'

### Load Data

In [3]:
def load_data():
    # Load data from csv and reset the index
    game_data = pd.read_csv(path.join(DATA_DIR,'scraped_nba_combined_2021_compressed.csv'))
    game_data = game_data.set_index('Index')

    # Convert possible rows to numeric values
    for col in game_data.columns:
        try:
            game_data[col] = game_data.to_numeric(df[col])
        except:
            continue

    return game_data

In [4]:
game_data = load_data()

latest_date = str(game_data.loc[game_data.index[len(game_data.index) - 1], 'Date'])

year = int(latest_date[0:4])
month = int(latest_date[4:6])
day = int(latest_date[6:8])

toall = date.today()
toyear = int(toall.strftime("%Y"))
tomonth = int(toall.strftime("%m"))
today = int(toall.strftime("%d"))

if toyear >= year and (tomonth > month or (tomonth == month and today > day + 1)):
    os.system('python update_nba_boxscores_compressed.py')
    os.system('python update_nba_odds_compressed.py')
    os.system('python combine_boxscore_odds_compressed.py')

    game_data = load_data()

In [5]:
# # Days in each month
# days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]


# # Set today to yesterday
# if today == 1:
#     if tomonth == 1:
#         tomonth = 12
#         toyear -= 1
#     else:
#         tomonth -= 1
#     today = days[tomonth]
# else:
#     today -= 1


# # Allow date override
# date_input = input("Date to grade in YYYYMMDD format (press ENTER to default to yesterday)")

# if date_input != None and date_input != '' and not date_input.isspace():
#     year = date_input[0:4]
#     month = date_input[4:6]
#     day = date_input[6:8]
# else:
#     date_input = f'{toyear}{tomonth:02d}{today:02d}'

# date_input = int(date_input)
# date_input

In [6]:
predictions = pd.read_csv(path.join(DATA_DIR, f'daily_predictions\\daily_predictions.csv'))
predictions = predictions.set_index('Index')


# Update only ungraded predictions
inds = list(predictions.loc[predictions['Spread Correct'].isnull()].index)
# Update all prediction grades
#inds = list(predictions.index)
inds = [ind for ind in inds if ind in list(game_data.index)]

for ind in inds:
    spread = 'Home' if predictions.loc[ind, 'Spread Prediction'] == predictions.loc[ind, 'Home Team'] else 'Away'
    not_spread = 'Home' if spread != 'Home' else 'Away'
    if game_data.loc[ind, f'{spread} T'] - game_data.loc[ind, f'{not_spread} T'] > -predictions.loc[ind, f'{spread} Spread']:
        predictions.loc[ind, 'Spread Correct'] = 'Yes'
    else:
        predictions.loc[ind, 'Spread Correct'] = 'No'

    total = predictions.loc[ind, 'Total Prediction']
    if (total == 'Over' and game_data.loc[ind, 'Home T'] + game_data.loc[ind, 'Away T'] > predictions.loc[ind, 'Over']) or (total == 'Under' and game_data.loc[ind, 'Home T'] + game_data.loc[ind, 'Away T'] < predictions.loc[ind, 'Over']):
        predictions.loc[ind, 'Total Correct'] = 'Yes'
    else:
        predictions.loc[ind, 'Total Correct'] = 'No'

    ml = 'Home' if predictions.loc[ind, 'ML Prediction'] == predictions.loc[ind, 'Home Team'] else 'Away'
    not_ml = 'Home' if ml != 'Home' else 'Away'
    if game_data.loc[ind, f'{ml} T'] > game_data.loc[ind, f'{not_ml} T']:
        predictions.loc[ind, 'ML Correct'] = 'Yes'
    else:
        predictions.loc[ind, 'ML Correct'] = 'No'

predictions.style.apply(lambda y: ['background: green' if v == 'Yes' else 'background: red' if v == 'No' else '' for v in y], axis = 1, subset = ['Spread Correct', 'Total Correct', 'ML Correct'])

predictions.to_csv(path.join(DATA_DIR, f'daily_predictions\\daily_predictions.csv'))

,Date,Home Team,Away Team,Home Spread,Home Spread Odds,Away Spread,Away Spread Odds,Home ML,Away ML,Over,Over Odds,Under,Under Odds,Spread Prediction,Spread Confidence,Total Prediction,Total Confidence,ML Prediction,ML Confidence,Spread Correct,Total Correct,ML Correct
Index,,,,,,,,,,,,,,,,,,,,,,
20210223_ATL_CLE,20210223,ATL,CLE,-8.000000,-103.000000,8.000000,-107.000000,-315.000000,285.000000,228.000000,-108.000000,228.000000,-102.000000,ATL,4,Over,2,ATL,4,No,No,No
20210223_DET_ORL,20210223,DET,ORL,4.500000,-107.000000,-4.500000,-103.000000,160.000000,-170.000000,211.000000,-101.000000,211.000000,-109.000000,DET,3,Over,3,DET,2,Yes,No,Yes
20210223_GSW_NYK,20210223,GSW,NYK,-4.000000,-108.000000,4.000000,-102.000000,-155.000000,145.000000,222.500000,-108.000000,222.500000,-102.000000,GSW,1,Under,5,GSW,3,Yes,Yes,Yes
20210223_PHI_TOR,20210223,PHI,TOR,-2.500000,-108.000000,2.500000,-102.000000,-140.000000,130.000000,222.000000,-105.000000,222.000000,-105.000000,PHI,5,Over,3,PHI,5,Yes,No,Yes
20210223_BOS_DAL,20210223,BOS,DAL,-2.000000,-109.000000,2.000000,-101.000000,-125.000000,115.000000,224.000000,-105.000000,224.000000,-105.000000,BOS,1,Over,3,BOS,2,No,No,No
20210223_SAC_BRK,20210223,SAC,BRK,6.000000,-101.000000,-6.000000,-109.000000,220.000000,-240.000000,242.500000,-105.000000,242.500000,-105.000000,BRK,4,Over,1,BRK,5,Yes,Yes,Yes
20210223_MIN_MIL,20210223,MIN,MIL,10.000000,-102.000000,-10.000000,-108.000000,480.000000,-570.000000,232.000000,-103.000000,232.000000,-107.000000,MIL,3,Over,3,MIL,4,Yes,Yes,Yes
20210223_POR_DEN,20210223,POR,DEN,6.000000,-101.000000,-6.000000,-109.000000,215.000000,-235.000000,232.000000,-107.000000,232.000000,-103.000000,DEN,1,Over,3,POR,1,No,No,No
20210223_WAS_LAC,20210223,WAS,LAC,12.000000,-105.000000,-12.000000,-105.000000,660.000000,-840.000000,236.500000,-102.000000,236.500000,-108.000000,LAC,2,Under,2,LAC,5,Yes,No,Yes


In [7]:
correction_cols = ['Spread Correct', 'Total Correct', 'ML Correct']
#correction_cols = ['Spread Correct', 'Total Correct']

correct = sum(predictions[correction_cols].isin(['Yes']).sum(axis = 0))
incorrect = sum(predictions[correction_cols].isin(['No']).sum(axis = 0))
total = len(predictions[correction_cols]) * len(correction_cols)

pd.DataFrame({'Correct': [correct], 'Incorrect': [incorrect], 'Total': [total], 'Percent Correct': [correct / total]})

,Correct,Incorrect,Total,Percent Correct
0,65,58,123,0.528455


In [8]:
predictions.loc[predictions['Date'] == 20210303].style.apply(lambda y: ['background: green' if v == 'Yes' else 'background: red' if v == 'No' else '' for v in y], axis = 1, subset = ['Spread Correct', 'Total Correct', 'ML Correct'])

,Date,Home Team,Away Team,Home Spread,Home Spread Odds,Away Spread,Away Spread Odds,Home ML,Away ML,Over,Over Odds,Under,Under Odds,Spread Prediction,Spread Confidence,Total Prediction,Total Confidence,ML Prediction,ML Confidence,Spread Correct,Total Correct,ML Correct
Index,,,,,,,,,,,,,,,,,,,,,,
20210303_DET_TOR,20210303,DET,TOR,7.500000,-105.000000,-7.500000,-105.000000,265.000000,-295.000000,210.500000,-105.000000,210.500000,-105.000000,DET,3,Under,3,TOR,3,Yes,No,No
20210303_IND_CLE,20210303,IND,CLE,-7.000000,-105.000000,7.000000,-105.000000,-250.000000,230.000000,219.000000,-105.000000,219.000000,-105.000000,CLE,3,Over,4,CLE,1,Yes,Yes,No
20210303_UTA_PHI,20210303,UTA,PHI,-3.500000,-103.000000,3.500000,-107.000000,-150.000000,140.000000,230.000000,-105.000000,230.000000,-105.000000,UTA,3,Over,4,UTA,4,No,Yes,No
20210303_BRK_HOU,20210303,BRK,HOU,-10.000000,-105.000000,10.000000,-105.000000,-460.000000,410.000000,226.500000,-105.000000,226.500000,-105.000000,BRK,4,Under,4,BRK,5,Yes,No,Yes
20210303_CHI_NOP,20210303,CHI,NOP,6.000000,-109.000000,-6.000000,-101.000000,210.000000,-230.000000,237.000000,-102.000000,237.000000,-108.000000,NOP,2,Over,4,NOP,5,No,Yes,No
20210303_ATL_ORL,20210303,ATL,ORL,-3.000000,-105.000000,3.000000,-105.000000,-145.000000,135.000000,219.000000,-105.000000,219.000000,-105.000000,ATL,2,Over,4,ATL,3,No,Yes,Yes
20210303_CHO_MIN,20210303,CHO,MIN,-3.000000,-102.000000,3.000000,-108.000000,-140.000000,130.000000,234.500000,-105.000000,234.500000,-105.000000,CHO,5,Under,3,CHO,5,Yes,No,Yes
20210303_OKC_DAL,20210303,OKC,DAL,4.500000,-107.000000,-4.500000,-103.000000,167.000000,-178.000000,220.000000,-108.000000,220.000000,-102.000000,OKC,2,Under,5,OKC,1,No,Yes,No
20210303_GSW_POR,20210303,GSW,POR,-1.500000,-108.000000,1.500000,-102.000000,-117.000000,107.000000,233.000000,-107.000000,233.000000,-103.000000,GSW,4,Over,3,GSW,5,No,No,No
